In [2]:
import sys
import librosa
import numpy as np
import pandas as pd
import pydub
import sklearn as sk
from numpy import *
from librosa import display
import pickle

In [ ]:
def load_audio(path):
    audio = pydub.AudioSegment.from_file(path).set_frame_rate(44100).set_channels(1)
    audio = audio
    while(len(audio)<10000):
        audio=audio+audio
    raw = (np.fromstring(audio._data, dtype="int16") + 0.5) / (0x7FFF + 0.5)   # convert to float
#     raw=(raw-(raw.mean()))/raw.std()
    return raw

In [ ]:
def generate_spec2(path, hires=True , test=False):
    hires_ext = '.hires' if hires else ''
    
    audio = load_audio(path)
    spec = librosa.feature.melspectrogram(audio, sr=44100, n_fft=2205, hop_length=441, n_mels=180,
                                          fmax=16000)
    freqs = librosa.core.mel_frequencies(n_mels=180, fmax=16000)
    spec = librosa.core.perceptual_weighting(spec, freqs)#, ref_power=np.max)
    t=np.shape(spec)[1]
    i=1
    spec1 = spec[5:175,(i-1)*1000:i*1000]
#         spec1 += 40
#         spec1 /= 10.0
#     print(path+ str(i))
    spec1=(spec1-(spec1.mean()))/spec1.std()
    return spec1
    

In [ ]:
# bird_audio_path=sys.argv[1]
bird_audio_path="D:/iitd/SEM6/COP315/test/My recording 1.wav"
bird_audio_spec=generate_spec2(bird_audio_path)


In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical
model = Sequential()
model.add(Conv2D(128, kernel_size=(170,20), activation='relu', input_shape=(170,1000,1)))
model.add(MaxPooling2D(pool_size=(1,981)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(17, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
# model.summary()

In [ ]:
bird_audio_spec_np = np.asarray(bird_audio_spec)
bird_audio_spec_np = bird_audio_spec_np.reshape(1,170,1000,1)

In [ ]:
model.load_weights("16_classes_xeno.hdf5")


In [14]:
y_pred=model.predict(bird_audio_spec_np)
yp=np.argmax(y_pred,axis=1)

In [3]:
with open("D:/iitd/SEM6/COP315/bird_class_dict.pickle", 'rb') as handle:
    b = pickle.load(handle)

In [4]:
b.values()

dict_values(['alexandrine_parakeet', 'brahamini_myna', 'collared_dove', 'common_myna', 'hoopoe', 'house_crow', 'indian_peafowl', 'indian_robin', 'jungle_babbler', 'koel', 'red_vented_bulbul', 'red_whiskered_bulbul', 'rock_dove', 'rose-ringed_parakeet', 'tailor_bird', 'yellow_footed_pigeon'])

In [12]:
print(b[yp[0]])

rose-ringed_parakeet
